# Linear Regression

**References:**
- https://www.coursera.org/learn/guided-tour-machine-learning-finance/notebook/7Z9sN/linear-regression

## Least-Squares solution

Normal equation

\begin{equation}
    \mathbf{\hat{\beta}}
    =
    \left( \mathbf{X}^{T} \mathbf{X} \right)^{-1} \mathbf{X}^{T} \mathbf{y}
\end{equation}

## Exercise

### Linear Regression on data with linear features

\begin{equation}
    y(x)
    =
    a + b_1 \cdot X_1 + b_2 \cdot X_2 + b_3 \cdot X_3 + \sigma \cdot \varepsilon
\end{equation}

where $ \varepsilon \sim N(0, 1) $ is a Gaussian noise, and $ \sigma $ is its volatility, 
with the following choice of parameters:

- $ a = 1.0 $

- $ b_1, b_2, b_3 = (0.5, 0.2, 0.1) $

- $ \sigma = 0.1 $

- $ X_1, X_2, X_3 $ will be uniformally distributed in $ [-1,1] $

### Linear Regression on data with non-linear features

\begin{equation}
    y(x)
    =
    a + w_{00} \cdot X_1 + w_{01} \cdot X_2 + w_{02} \cdot X_3 + + w_{10} \cdot X_1^2 
+ w_{11} \cdot X_2^2 + w_{12} \cdot X_3^2 +  \sigma \cdot \varepsilon
\end{equation}

where

- $ w = [[1.0, 0.5, 0.2],[0.5, 0.3, 0.15]]  $

- and the rest of parameters is as above, with the same values of $ X_i $

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import tensorflow as tf
from tensorflow.python.layers import core as core_layers
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
tf.__version__

'1.5.0'

In [3]:
def reset_graph(seed=42):
    """
    Utility function to reset current tensorflow computation graph
    and set the random seed 
    """
    # to make results reproducible across runs
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [4]:
def generate_data(n_points=10000, n_features=3, use_nonlinear_data=True, 
                    noise_std=0.1, train_test_split = 4):
    """
    Arguments:
    n_points - number of data points to generate
    n_features - a positive integer - number of features
    use_nonlinear - if True, generate non-linear data
    train_test_split - an integer - what portion of data to use for testing
    
    Return:
    X_train, Y_tLinearrain, X_test, Y_test, n_train, n_features
    """
    
    # Linear data or non-linear data?
    if use_nonlinear_data:
        weights = np.array([[1.0, 0.5, 0.2],[0.5, 0.3, 0.15]])
    else:
        weights = np.array([1.0, 0.5, 0.2])
        
    
    bias =   np.ones(n_points).reshape((-1,1))
    low  = - np.ones((n_points,n_features),'float')
    high =   np.ones((n_points,n_features),'float')
        
    np.random.seed(42)
    X = np.random.uniform(low=low, high=high)
    
    np.random.seed(42)
    noise = np.random.normal(size=(n_points, 1))
    noise_std = 0.1
    
    if use_nonlinear_data:
        Y = (weights[0,0] * bias + np.dot(X, weights[0, :]).reshape((-1,1)) + 
             np.dot(X*X, weights[1, :]).reshape([-1,1]) +
             noise_std * noise)
    else:
        Y = (weights[0] * bias + np.dot(X, weights[:]).reshape((-1,1)) + 
             noise_std * noise)
    
    n_test = int(n_points/train_test_split)
    n_train = n_points - n_test
    
    X_train = X[:n_train,:]
    Y_train = Y[:n_train].reshape((-1,1))

    X_test = X[n_train:,:]
    Y_test = Y[n_train:].reshape((-1,1))
    
    return X_train, Y_train, X_test, Y_test, n_train, n_features

In [5]:
X_train, Y_train, X_test, Y_test, n_train, n_features = generate_data(use_nonlinear_data=False)
X_train.shape, Y_train.shape

((7500, 3), (7500, 1))

In [6]:
X_train = np.c_[np.ones(len(X_train)) , X_train ]
X_test  = np.c_[np.ones(len(X_test)) ,   X_test ]

## Linear regression with `numpy`

In [7]:
betahat_np = np.matmul(np.matmul( np.linalg.inv(np.matmul(X_train.T,X_train)), X_train.T), Y_train)
betahat_np

array([[0.99946227],
       [0.99579039],
       [0.499198  ],
       [0.20019798]])

In [8]:
betahat_np = np.linalg.inv(X_train.T.dot(X_train)).dot(X_train.T).dot(Y_train)
betahat_np

array([[0.99946227],
       [0.99579039],
       [0.499198  ],
       [0.20019798]])

Test

In [9]:
X_train.shape

(7500, 4)

In [10]:
X_train.T.shape

(4, 7500)

In [11]:
X_train.T @ X_train

array([[ 7.50000000e+03,  7.63529630e+01,  1.10196223e+01,
        -4.91540004e+01],
       [ 7.63529630e+01,  2.46507830e+03,  6.49185120e+00,
         2.66539829e+01],
       [ 1.10196223e+01,  6.49185120e+00,  2.53214517e+03,
        -2.81560831e+01],
       [-4.91540004e+01,  2.66539829e+01, -2.81560831e+01,
         2.51282096e+03]])

In [12]:
X_train

array([[ 1.        , -0.25091976,  0.90142861,  0.46398788],
       [ 1.        ,  0.19731697, -0.68796272, -0.68801096],
       [ 1.        , -0.88383278,  0.73235229,  0.20223002],
       ...,
       [ 1.        , -0.41969273,  0.7395476 ,  0.49515624],
       [ 1.        , -0.15112479,  0.42105835,  0.73537775],
       [ 1.        ,  0.95130426,  0.90470301, -0.34934435]])

## Linear Regression with `sklearn`

In [13]:
help(LinearRegression)

Help on class LinearRegression in module sklearn.linear_model.base:

class LinearRegression(LinearModel, sklearn.base.RegressorMixin)
 |  Ordinary least squares Linear Regression.
 |  
 |  Parameters
 |  ----------
 |  fit_intercept : boolean, optional, default True
 |      whether to calculate the intercept for this model. If set
 |      to False, no intercept will be used in calculations
 |      (e.g. data is expected to be already centered).
 |  
 |  normalize : boolean, optional, default False
 |      This parameter is ignored when ``fit_intercept`` is set to False.
 |      If True, the regressors X will be normalized before regression by
 |      subtracting the mean and dividing by the l2-norm.
 |      If you wish to standardize, please use
 |      :class:`sklearn.preprocessing.StandardScaler` before calling ``fit`` on
 |      an estimator with ``normalize=False``.
 |  
 |  copy_X : boolean, optional, default True
 |      If True, X will be copied; else, it may be overwritten.
 | 

In [14]:
lr = LinearRegression(fit_intercept=False)

In [15]:
lr.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None,
         normalize=False)

In [16]:
betahat_sklearn = lr.coef_
betahat_sklearn

array([[0.99946227, 0.99579039, 0.499198  , 0.20019798]])

## Linear Regression with `tensorflow`

In [17]:
X_train_tf = tf.constant(X_train, dtype=tf.float32, name="X_train")
Y_train_tf = tf.constant(Y_train, dtype=tf.float32, name="Y_train")

In [18]:
X_train_tf_T = tf.transpose(X_train_tf)

In [19]:
betahat_tf = tf.matmul(tf.matmul(tf.matrix_inverse( tf.matmul(X_train_tf_T, X_train_tf)), X_train_tf_T), Y_train_tf)

In [20]:
with tf.Session() as sess:
    betahat_tf_value = betahat_tf.eval()

In [21]:
betahat_tf_value

array([[0.99946195],
       [0.99579084],
       [0.49919814],
       [0.20019816]], dtype=float32)

----------------

In [22]:
class LinearRegressionModel:

    def __init__(self, n_features, learning_rate=0.05, L=0):
                
        # input placeholders
        self.X = tf.placeholder(tf.float32, [None, n_features], name="X")
        self.Y = tf.placeholder(tf.float32, [None, 1], name="Y")
    
        # regression parameters for the analytical solution using the Normal equation
        self.theta_in = tf.placeholder(tf.float32, [n_features,None])

        # Augmented data matrix is obtained by adding a column of ones to the data matrix
        data_plus_bias = self.X
        #data_plus_bias = tf.concat([tf.ones([tf.shape(self.X)[0], 1]), self.X], axis=1)
        
        XT = tf.transpose(data_plus_bias)
        
        #############################################
        # The normal equation for Linear Regression
        
        self.theta = tf.matmul(tf.matmul(
            tf.matrix_inverse(tf.matmul(XT, data_plus_bias)), XT), self.Y)
        
        # mean square error in terms of theta = theta_in
        self.lr_mse = tf.reduce_mean(tf.square(
            tf.matmul(data_plus_bias, self.theta_in) - self.Y))
                       
        #############################################
        # Estimate the model using the Maximum Likelihood Estimation (MLE)
        
        # regression parameters for the Maximum Likelihood method
        # Note that there are n_features+2 parameters, as one is added for the intercept, 
        # and another one for the std of noise  
        self.weights = tf.Variable(tf.random_normal([n_features+1, 1]))
        
        # prediction from the model
        self.output = tf.matmul(data_plus_bias, self.weights[:-1, :])

        gauss = tf.distributions.Normal(loc=0.0, scale=1.0)

        # Standard deviation of the Gaussian noise is modelled as a square of the 
        # last model weight
        sigma = 0.0001 + tf.square(self.weights[-1]) 
        
        # though a constant sqrt(2*pi) is not needed to find the best parameters, here we keep it
        # to get the value of the log-LL right 
        pi = tf.constant(np.pi)
    
        log_LL = tf.log(0.00001 + (1/( tf.sqrt(2*pi)*sigma)) * gauss.prob((self.Y - self.output) / sigma ))  
        self.loss = - tf.reduce_mean(log_LL)
        
        self.train_step = (tf.train.AdamOptimizer(learning_rate).minimize(self.loss), -self.loss)

In [23]:
n_features = X_train.shape[1]
n_features

4

In [24]:
learning_rate=0.05

In [25]:
model = LinearRegressionModel(n_features=n_features, learning_rate=learning_rate)

In [26]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    theta_value  = sess.run(model.theta, 
                            feed_dict={
                                model.X: X_train,
                                model.Y: Y_train
                            })

    lr_mse_train = sess.run(model.lr_mse,
                            feed_dict={
                                model.X: X_train,
                                model.Y: Y_train,
                                model.theta_in: theta_value
                            })

    lr_mse_test = sess.run(model.lr_mse,
                            feed_dict={
                                model.X: X_test,
                                model.Y: Y_test,
                                model.theta_in: theta_value
                            })

------

In [28]:
num_iter=5000

In [29]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Now train the MLE parameters 
    for _ in range(num_iter):
        (_ , loss), weights = sess.run((model.train_step, model.weights), feed_dict={
            model.X: X_train,
            model.Y: Y_train
            })

    # make test_prediction
    Y_test_predicted = sess.run(model.output, feed_dict={model.X: X_test})

    # output std sigma is a square of the last weight
    std_model = weights[-1]**2 
    sess.close()

In [30]:
weights, loss, std_model

(array([[0.9969759 ],
        [0.9934216 ],
        [0.4961067 ],
        [0.20324217],
        [0.31699133]], dtype=float32),
 -0.041042022,
 array([0.1004835], dtype=float32))